In [2]:
#in this document we'll keep the demo chatgpt interactions


#### Ok Concept Demonstrated! woohoo the world is falling apart. 
#A little jank: two failure modes that I see are 
#.   poor difficulty scaling (sometimes a question is the same in two difficulty modes)
#.   poorly contained questions - references to "the text"
## these both are prompt engineering things- structuring the bloom's taxonomy prompt better will solve the first one.

#but ultimately this is a pretty delightful little output, I think.
# I need to do the SWENG stuff next- packing in the learning objectives as i described in the proposal paper
# then the whole recommendation system- that requires some thinking about how to store the KG. 
##### I get to do some digging on that :3
#I should worry about shoving KC context into the prompts a chapter-aware way. A chapter 1 term should not get chapter 21 context,
#unless the student is also looking at other chapter 21 content. 

#Then there's the evaluation engineering thing- a whole extra prompt loop running on actual user input. 



In [22]:
import pickle
import random
from knowledge_component import KC, ContentExample
from openai import OpenAI

client = OpenAI(
)

KG = pickle.load(open("sociology_kg.pickle", "rb"))

In [20]:
def term_to_prompt(term, objective="Knowledge"):
    if len(term.context) > 0:
        context = random.sample(term.context, 1)[0].full_text
    else:
        context = "there is no extra context"

    learning_objectives = {
        "Knowledge":"can recognize or remember facts, terms, basic concepts, or answers without necessarily understanding what they mean.",
        "Comprehend":"can demonstrate an understanding of facts and ideas by organizing, summarizing, translating, generalizing, giving descriptions, and stating the main ideas. ",
        "Apply":"can use the term correctly in new circumstances",
        "Analyze":"can examine and break information into component parts, determine how the parts relate to one another, identify motives or causes, make inferences, and find evidence to support generalizations"
    }
    system_prompt = f"""
You are the voice of an intelligent tutoring system made to help students learn sociology. The user provides you a term, its definition, and a paragraph of context from
their textbook. You generate only a question/answer pair in json format targeting this content. Consider your answer a possible reference answer to guide how the question is posed.
The question should primarily target the given term, and the additional context can be used to guide or supplement the question at higher difficulties. 
You will also be given a learning objective from Bloom's Taxonomy- one of "Knowledge", "Comprehend", "Apply" and "Analyse"- each indicating a more sophisticated question than the prior objective
You will scale the difficulty of the question you ask based on this objective. The question should not directly reference any exterior content. 
"""
    
    user_prompt = f"""
The student has chosen to recieve a question pertaining to the following term term: {term.title}
The students textbook defines as: {term.definition}
This question will pertain to the content of the following text: {context}


In this case, the learning objective is "{objective}"- your question will help the student demonstrate that they {learning_objectives[objective]}
    """
    return system_prompt, user_prompt

In [24]:
term = random.choice(list(KG["chapter 1"]))
print(term)


def ask_gpt(term, lo):
    
    system, user = term_to_prompt(term, lo)
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": system},
        {"role": "user", "content": user}
      ]
    )
    
    return response.choices[0].message.content

print(ask_gpt(term, "Knowledge"))
print(ask_gpt(term, "Comprehend"))
print(ask_gpt(term, "Apply"))
print(ask_gpt(term, "Analyze"))

positivism : the scientific study...
[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n\t"question": "According to the provided context, which famous theorist rejected Comte\'s positivism and instead believed that societies grew and changed due to the struggles of different social classes over the means of production?",\n\t"answer": "Karl Marx rejected Comte\'s positivism and proposed that social change occurs through the conflicts between social classes over the means of production."\n}', role='assistant', function_call=None, tool_calls=None))]
{
	"question": "According to the provided context, which famous theorist rejected Comte's positivism and instead believed that societies grew and changed due to the struggles of different social classes over the means of production?",
	"answer": "Karl Marx rejected Comte's positivism and proposed that social change occurs through the conflicts between social classes over the means of production."
}


In [25]:
test = ask_gpt(term, "Knowledge")

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "question": "What term describes the scientific study of social patterns?",\n  "answer": "Positivism"\n}', role='assistant', function_call=None, tool_calls=None))]


In [26]:
test

'{\n  "question": "What term describes the scientific study of social patterns?",\n  "answer": "Positivism"\n}'

In [27]:
import json
json.loads(test)

{'question': 'What term describes the scientific study of social patterns?',
 'answer': 'Positivism'}